In [1]:
import pandas as pd
import numpy as np

In [2]:
# 화면에 출력하는 데이터 프레임의 최대 row 수를 500으로 설정합니다.
pd.set_option('display.max_rows', 500)

# 화면에 출력하는 데이터 프레임의 최대 column 수를 500으로 설정합니다.
pd.set_option('display.max_columns', 500)

In [3]:
import matplotlib
import matplotlib.pyplot as plt  # 파이플롯 사용
import seaborn as sns
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')  # 한글코드를 더 선명하게 해주는 조치, 레티나 설정
matplotlib.rc('font', family='AppleGothic') # 폰트 설정
matplotlib.rc('axes', unicode_minus=False) # 마이너스 폰트가 깨지는 경우가 있으므로 조치

In [4]:
df = pd.read_csv("전라남도_축제별방문객현황.csv", encoding='euc-kr', engine='python')

In [5]:
df = df.drop(['연번', '시도명', '주최/주관', '담당자 연락처', '개최장소', '데이터기준일자', 'Unnamed: 16'], axis=1)

In [6]:
df.columns = ['district', 'fest_name', 'fest_19_month', 'contents', '17_total', '17_kor', '17_for', '18_total', '18_kor', '18_for']


### 전처리1. 19년 축제 개최일을 개최월로 변경

In [7]:
def festival_month(date):
    date1 = date.split('/')[0]
    
    if '월' in(date1):
        return int(date1.split('월')[0])
    if '가을' in(date1):
        return 10
    else:
        return int(list(str(date1))[0])

In [8]:
df['fest_19_month'] = df['fest_19_month'].apply(festival_month)

### 전처리2. 17년, 18년 참가자 숫자처리

*17년은 1000명 단위이고, 18년은 1명 단위라는 걸 명심하자*

- 외국인 파트가 nan 이면 0으로 처리한다
- 내국인 파트가 nan 이면 해당 연도의 데이터는 폐기한다 (모두 nan 처리)

In [9]:
df.loc[df['17_for'].isnull(), '17_for'] = 0

In [10]:
df.loc[df['17_kor'].isnull(), ['17_total','17_kor','17_for']] = 0

In [11]:
df.loc[df['18_for'].isnull(), '18_for'] = 0

In [12]:
df.loc[df['18_kor'].isnull(), ['18_total','18_kor','18_for']] = 0

In [13]:
df.iloc[105, 9] = 0
df.iloc[106, 9] = 0

In [14]:
df['18_for'] = df['18_for'].astype(int)

In [15]:
df['18_kor'] = (df['18_kor']/1000).astype(int)
df['18_for'] = (df['18_for']/1000).astype(int)


In [17]:
df['17_kor'] = df['17_kor'].astype(int)
df['17_for'] = df['17_for'].astype(int)

In [18]:
df = df.drop(['17_total', '18_total'], axis=1)
df['17_total'] = df['17_kor'] + df['17_for']
df['18_total'] = df['18_kor'] + df['18_for']

In [25]:
df_normal = df.loc[(df['17_total'] & df['18_total']) > 0]
df_normal.shape

(77, 10)